In [1]:
!pip install transformers

In [3]:
import torch
from transformers import BertForQuestionAnswering
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [5]:
import textwrap
wrapping = textwrap.TextWrapper(width=80)

In [8]:
text = """
Sunita Lyn "Suni" Williams (née Pandya; born September 19, 1965) is an Indian-American astronaut, commander of the International Space Station, retired U.S. Navy officer, and one of the most experienced spacewalkers with nine spacewalks (second most for a woman) and a total time of 62 hours and 6 minutes (fourth overall, current record-holder for most hours spent during spacewalks by a woman). Williams was assigned to the International Space Station as a member of Expedition 14 and Expedition 15. In 2012, she served as a flight engineer on Expedition 32 and then commander of Expedition 33.


Williams was born in Euclid, Ohio, but considers Needham, Massachusetts, to be her hometown.[1] Her father, Deepak Pandya, was an Indian-American neuroanatomist from Mehsana district, Gujarat, while her mother, Ursuline Bonnie Pandya (née Zalokar) was a Slovene-American. Her parents resided in Falmouth, Massachusetts. She is the youngest of three children. Her brother, Jay Thomas, is four years older and her sister, Dina Annad, is three years older. Williams' paternal family is from Jhulasan in India[2] whereas her maternal family is of Slovene descent.[3] Williams has taken the Slovenian flag,[4] a samosa and Carniolan sausage to space in celebration of her Indian and Slovenian heritage.[5] Her nicknames are Suni in the United States[1] and Sončka in Slovenia. the United States Naval Academy in 1987, and a Master of Science degree in engineering management from Florida Institute of Technology in 1995.

Williams graduated from Needham High School in 1983."""

In [9]:
text

'\nSunita Lyn "Suni" Williams (née Pandya; born September 19, 1965) is an Indian-American astronaut, commander of the International Space Station, retired U.S. Navy officer, and one of the most experienced spacewalkers with nine spacewalks (second most for a woman) and a total time of 62 hours and 6 minutes (fourth overall, current record-holder for most hours spent during spacewalks by a woman). Williams was assigned to the International Space Station as a member of Expedition 14 and Expedition 15. In 2012, she served as a flight engineer on Expedition 32 and then commander of Expedition 33.\n\n\nWilliams was born in Euclid, Ohio, but considers Needham, Massachusetts, to be her hometown.[1] Her father, Deepak Pandya, was an Indian-American neuroanatomist from Mehsana district, Gujarat, while her mother, Ursuline Bonnie Pandya (née Zalokar) was a Slovene-American. Her parents resided in Falmouth, Massachusetts. She is the youngest of three children. Her brother, Jay Thomas, is four yea

In [10]:
wrapped_text = wrapping.fill(text)
print(wrapped_text)

 Sunita Lyn "Suni" Williams (née Pandya; born September 19, 1965) is an Indian-
American astronaut, commander of the International Space Station, retired U.S.
Navy officer, and one of the most experienced spacewalkers with nine spacewalks
(second most for a woman) and a total time of 62 hours and 6 minutes (fourth
overall, current record-holder for most hours spent during spacewalks by a
woman). Williams was assigned to the International Space Station as a member of
Expedition 14 and Expedition 15. In 2012, she served as a flight engineer on
Expedition 32 and then commander of Expedition 33.   Williams was born in
Euclid, Ohio, but considers Needham, Massachusetts, to be her hometown.[1] Her
father, Deepak Pandya, was an Indian-American neuroanatomist from Mehsana
district, Gujarat, while her mother, Ursuline Bonnie Pandya (née Zalokar) was a
Slovene-American. Her parents resided in Falmouth, Massachusetts. She is the
youngest of three children. Her brother, Jay Thomas, is four years o

In [11]:
import re
cleaned_text =  re.sub(r"\[\d+\]","",wrapped_text)

In [12]:
wrapped_text

' Sunita Lyn "Suni" Williams (née Pandya; born September 19, 1965) is an Indian-\nAmerican astronaut, commander of the International Space Station, retired U.S.\nNavy officer, and one of the most experienced spacewalkers with nine spacewalks\n(second most for a woman) and a total time of 62 hours and 6 minutes (fourth\noverall, current record-holder for most hours spent during spacewalks by a\nwoman). Williams was assigned to the International Space Station as a member of\nExpedition 14 and Expedition 15. In 2012, she served as a flight engineer on\nExpedition 32 and then commander of Expedition 33.   Williams was born in\nEuclid, Ohio, but considers Needham, Massachusetts, to be her hometown.[1] Her\nfather, Deepak Pandya, was an Indian-American neuroanatomist from Mehsana\ndistrict, Gujarat, while her mother, Ursuline Bonnie Pandya (née Zalokar) was a\nSlovene-American. Her parents resided in Falmouth, Massachusetts. She is the\nyoungest of three children. Her brother, Jay Thomas, is

In [13]:
def answer_question(question,answer_text):
  input_ids=tokenizer.encode(question,answer_text)
  print("Query has {:,} tokens.\n".format(len(input_ids)))
  sep_index = input_ids.index(tokenizer.sep_token_id)
  num_seg_a = sep_index +1
  num_seg_b= len(input_ids)- num_seg_a
  segment_ids = [0]*num_seg_a + [1]*num_seg_b
  assert len(segment_ids)== len(input_ids)
  outputs = model(torch.tensor([input_ids]),token_type_ids=torch.tensor([segment_ids]),return_dict=True)
  start_scores = outputs.start_logits
  end_scores = outputs.end_logits
  answer_start = torch.argmax(start_scores)
  answer_end = torch.argmax(end_scores)
  tokens = tokenizer.convert_ids_to_tokens(input_ids)
  answer = tokens[answer_start]
  for i in range(answer_start+1,answer_end+1):
    if tokens[i][0:2]=="##":
       answer+= tokens[i][2:]
    else:
      answer += " " + tokens[i]
  print('Answer: "' + answer + '"')


In [14]:
question = "Who is Sunita Lyn Williams?"
answer_question(question,cleaned_text)

Query has 352 tokens.

Answer: "an indian - american astronaut"


In [15]:
question = "Where and when was Sunita Williams born?"
answer_question(question,cleaned_text)

Query has 353 tokens.

Answer: "september 19 , 1965"


In [16]:
question = "What is Sunita Williams ethnic background?"
answer_question(question,cleaned_text)

Query has 352 tokens.

Answer: "indian - american"


In [17]:
question ="What are Sunita Williams' nicknames, and what do they mean?"
answer_question(question,cleaned_text)

Query has 359 tokens.

Answer: "suni in the united states and soncka in slovenia"


In [18]:
question = "What is Sunita Williams known for in her career?"
answer_question(question,cleaned_text)


Query has 355 tokens.

Answer: "commander of the international space station"


In [20]:
question = "How many spacewalks has Sunita Williams completed?"
answer_question(question,cleaned_text)

Query has 355 tokens.

Answer: "nine"


In [21]:
question ="What is Sunita Williams' total time spent on spacewalks?"
answer_question(question,cleaned_text)

Query has 358 tokens.

Answer: "62 hours and 6 minutes"


In [22]:
question = "Which expeditions did Sunita Williams participate in on the International Space Station?"
answer_question(question,cleaned_text)

Query has 358 tokens.

Answer: "expedition 14 and expedition 15"


In [23]:
question = "What roles did Sunita Williams hold during Expeditions 32 and 33?"
answer_question(question,cleaned_text)

Query has 357 tokens.

Answer: "flight engineer"


In [24]:
question = "What records does Sunita Williams hold in space exploration?"
answer_question(question,cleaned_text)


Query has 355 tokens.

Answer: "nine spacewalks ( second most for a woman ) and a total time of 62 hours and 6 minutes"


In [25]:
question = "Where did Sunita Williams graduate from high school?"
answer_question(question,cleaned_text)

Query has 354 tokens.

Answer: "needham"


In [26]:
question = "What degrees does Sunita Williams hold, and from which institutions?"
answer_question(question,cleaned_text)

Query has 357 tokens.

Answer: "united states naval academy in 1987 , and a master of science degree in engineering management from florida institute of technology"


In [27]:
question = "What is Sunita Williams' connection to the U.S. Navy?"
answer_question(question,cleaned_text)


Query has 359 tokens.

Answer: "retired"


In [28]:
question = "What is Sunita Williams' family background?"
answer_question(question,cleaned_text)


Query has 353 tokens.

Answer: "williams ' paternal family is from jhulasan in india"


In [30]:
question = "What items did Sunita Williams take to space to celebrate her heritage?"
answer_question(question,cleaned_text)

Query has 358 tokens.

Answer: "slovenian flag , a samosa and carniolan sausage"


In [31]:
question = "What is the significance of the Slovenian flag, samosa, and Carniolan sausage to Sunita Williams?"
answer_question(question,cleaned_text)

Query has 366 tokens.

Answer: "celebration of her indian and slovenian heritage"


In [32]:
question = "What are some unique facts about Sunita Williams' life and career?"
answer_question(question,cleaned_text)

Query has 358 tokens.

Answer: "nine spacewalks ( second most for a woman ) and a total time of 62 hours and 6 minutes"


In [33]:
question = "How does Sunita Williams honor her Indian and Slovenian heritage in space?"
answer_question(question,cleaned_text)

Query has 358 tokens.

Answer: "taken the slovenian flag , a samosa and carniolan sausage"


In [34]:
question = "What are some of Sunita Williams' hobbies or interests outside of her career?"
answer_question(question,cleaned_text)

Query has 361 tokens.

Answer: "retired u . s . navy officer , and one of the most experienced spacewalkers with nine spacewalks ( second most for a woman ) and a total time of 62 hours and 6 minutes ( fourth overall , current record - holder for most hours spent during spacewalks by a woman ) . williams was assigned to the international space station as a member of expedition 14 and expedition 15 . in 2012 , she served as a flight engineer on expedition 32 and then commander of expedition 33 . williams was born in euclid , ohio , but considers needham , massachusetts , to be her hometown . her father , deepak pandya , was an indian - american neuroanatomist from mehsana district , gujarat , while her mother , ursuline bonnie pandya ( nee zalokar ) was a slovene - american . her parents resided in falmouth , massachusetts . she is the youngest of three children . her brother , jay thomas , is four years older and her sister , dina annad , is three years older . williams ' paternal fami